In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from features.settings import Baseline_Settings, Pair_Hinge_Settings, Pair_Log_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings
from features.models import ConvModelSmall, ConvModelMedium, ConvModelLarge
from features.models import GCNNModelSmall, GCNNModelMedium, GCNNModelLarge
from classes.models import LinearClassifier
from datasets import load_mnist_datasets, load_cifar10
import time

/home/dqi/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/dqi/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data, samplers, pair_samplers, triplet_samplers = load_mnist_datasets(augment=True)

MNIST loaded in 18 seconds
Fashion MNIST loaded in 35 seconds


In [3]:
def test_model(feature_model, settings, data, num_features, num_classes, feature_samplers, classification_samplers,
               f_num_steps, f_lr, f_keep_prob, f_reg, c_num_steps, c_lr, c_reg):
    
    tf.reset_default_graph()
    
    f_train, f_test = feature_samplers
    c_train, c_test = classification_samplers
    (x_train, y_train), (x_valid, y_valid), (x_test, y_test) = data
    _, H, W, C = x_train.shape
    
    x = tf.placeholder(tf.float32, shape=[None, H, W, C])
    y = tf.placeholder(tf.int64, shape=[None])    
    dropout = tf.placeholder(tf.float32)
    training = tf.placeholder(tf.bool)
    f_model = feature_model(x=x, y=y, settings=settings, num_chan=C, num_features=num_features, 
              lr=f_lr, reg=f_reg, dropout=dropout, training=training)
    
    features = tf.placeholder(tf.float32, shape=[None, num_features])
    c_model = LinearClassifier(x=features, y=y, num_features=num_features, num_classes=num_features, 
                               lr=c_lr, reg=c_reg)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        print("begin training features // num_features: %g, lr: %g, reg: %g, dropout: %g" %(num_features, f_lr, f_reg, f_keep_prob))
        train_time = time.time()
        for step in range(f_num_steps):        
            x_, y_ = f_train.sample(700)
            sess.run(f_model.optimize, feed_dict={x:x_, y:y_, dropout:f_keep_prob, training:True})     
            if step % 5000 == 0:
                train_loss, train_acc = sess.run([f_model.loss, f_model.acc], 
                                                 feed_dict={x:x_, y:y_, dropout:1.0, training:False})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training features // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        x_, y_ = f_test.sample(1000)
        test_error = 1 - sess.run(f_model.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        print("begin training classifier")
        train_time = time.time()
        for step in range(c_num_steps):        
            x_, y_ = c_train.sample(700)
            features_ = sess.run(f_model.features, feed_dict={x:x_, dropout:1.0, training:False})
            sess.run(c_model.optimize, feed_dict={features:features_, y:y_, x:x_, dropout:1.0, training:False})
            if step % 5000 == 0:
                train_loss, train_acc = sess.run([c_model.loss, c_model.acc], feed_dict={features:features_, y:y_})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training classifier // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        features_ = sess.run(f_model.features, feed_dict={x:x_valid, dropout:1.0, training:False})
        test_error = 1 - sess.run(c_model.acc, feed_dict={features:features_, y:y_valid})
        eval_test_time = time.time() - eval_test_time
        print("validation set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        eval_test_time = time.time()
        features_ = sess.run(f_model.features, feed_dict={x:x_test, dropout:1.0, training:False})
        test_error = 1 - sess.run(c_model.acc, feed_dict={features:features_, y:y_test})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))

# MNIST

In [9]:
test_model(feature_model=ConvModelSmall, settings=Baseline_Settings, num_features=10, num_classes=10,
           data=data['mnist'], feature_samplers=samplers['mnist'], classification_samplers=samplers['mnist'],           
           f_num_steps=50000, f_lr=0.07, f_keep_prob=0.8, f_reg=0.001, 
           c_num_steps=15000, c_lr=0.02, c_reg=0.001)

begin training features // num_features: 10, lr: 0.07, reg: 0.001, dropout: 0.8
	step 0: train loss 2.5844, train error 0.938571
	step 5000: train loss 0.074062, train error 0.00857145
	step 10000: train loss 0.0418246, train error 0.00428569
	step 15000: train loss 0.0426467, train error 0.00142854
	step 20000: train loss 0.0287257, train error 0
	step 25000: train loss 0.0292975, train error 0
	step 30000: train loss 0.0296211, train error 0.00142854
	step 35000: train loss 0.0286408, train error 0.00142854
	step 40000: train loss 0.0297514, train error 0.00285715
	step 45000: train loss 0.0268356, train error 0
end training features // time elapsed: 531.8733 s
test set error: 0.0020 // time elapsed: 0.0313 s
begin training classifier
	step 0: train loss 4.10524, train error 0.665714
	step 5000: train loss 0.0145175, train error 0.00285715
	step 10000: train loss 0.0129495, train error 0.00428569
end training classifier // time elapsed: 130.1927 s
validation set error: 0.0077 // time

In [6]:
test_model(feature_model=ConvModelSmall, settings=Triplet_Hinge_Settings, num_features=10, num_classes=10,
           data=data['mnist'], feature_samplers=triplet_samplers['mnist'], classification_samplers=samplers['mnist'],           
           f_num_steps=50000, f_lr=0.07, f_keep_prob=0.8, f_reg=0.001, 
           c_num_steps=20000, c_lr=0.005, c_reg=0.001)

begin training features // num_features: 10, lr: 0.07, reg: 0.001, dropout: 0.8
	step 0: train loss 1.22948, train error 0.48
	step 5000: train loss 0.0383013, train error 0
	step 10000: train loss 0.0156607, train error 0
	step 15000: train loss 0.015407, train error 0.00142854
	step 20000: train loss 0.0132827, train error 0
	step 25000: train loss 0.0132661, train error 0
	step 30000: train loss 0.0178738, train error 0.00142854
	step 35000: train loss 0.0144385, train error 0.00142854
	step 40000: train loss 0.0114285, train error 0
	step 45000: train loss 0.0109364, train error 0
end training features // time elapsed: 1444.4424 s
test set error: 0.0010 // time elapsed: 0.0544 s
begin training classifier
	step 0: train loss 2.69781, train error 0.872857
	step 5000: train loss 0.0695118, train error 0.0057143
	step 10000: train loss 0.0475809, train error 0.00285715
	step 15000: train loss 0.0643878, train error 0.00999999
end training classifier // time elapsed: 173.2038 s
validati

In [5]:
test_model(feature_model=GCNNModelSmall, settings=Baseline_Settings, num_features=10, num_classes=10,
           data=data['mnist'], feature_samplers=samplers['mnist'], classification_samplers=samplers['mnist'],           
           f_num_steps=60000, f_lr=0.04, f_keep_prob=0.8, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.001)

begin training features // num_features: 10, lr: 0.04, reg: 0.001, dropout: 0.8
	step 0: train loss 2.79066, train error 0.931429
	step 5000: train loss 0.169144, train error 0.00142854
	step 10000: train loss 0.0591267, train error 0
	step 15000: train loss 0.0287463, train error 0
	step 20000: train loss 0.0199411, train error 0
	step 25000: train loss 0.0171866, train error 0
	step 30000: train loss 0.0155183, train error 0
	step 35000: train loss 0.0146768, train error 0
	step 40000: train loss 0.0143286, train error 0
	step 45000: train loss 0.0136115, train error 0
	step 50000: train loss 0.0131566, train error 0
	step 55000: train loss 0.0127228, train error 0
end training features // time elapsed: 2341.1535 s
test set error: 0.0000 // time elapsed: 0.0467 s
begin training classifier
	step 0: train loss 6.2264, train error 0.921429
	step 5000: train loss 0.00818373, train error 0
	step 10000: train loss 0.00659444, train error 0
	step 15000: train loss 0.00573866, train error 0


In [4]:
test_model(feature_model=GCNNModelSmall, settings=Triplet_Hinge_Settings, num_features=10, num_classes=10,
           data=data['mnist'], feature_samplers=triplet_samplers['mnist'], classification_samplers=samplers['mnist'],           
           f_num_steps=60000, f_lr=0.04, f_keep_prob=0.8, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.001)

begin training features // num_features: 10, lr: 0.04, reg: 0.001, dropout: 0.8
	step 0: train loss 1.49302, train error 0.427143
	step 5000: train loss 0.155671, train error 0.00142854
	step 10000: train loss 0.0482894, train error 0
	step 15000: train loss 0.0187268, train error 0
	step 20000: train loss 0.0104779, train error 0
	step 25000: train loss 0.00810195, train error 0
	step 30000: train loss 0.00734568, train error 0
	step 35000: train loss 0.00700255, train error 0
	step 40000: train loss 0.00678346, train error 0
	step 45000: train loss 0.00663847, train error 0
	step 50000: train loss 0.00653485, train error 0
	step 55000: train loss 0.00787598, train error 0
end training features // time elapsed: 6788.8070 s
test set error: 0.0010 // time elapsed: 0.1521 s
begin training classifier
	step 0: train loss 2.6431, train error 0.974286
	step 5000: train loss 0.0577013, train error 0.00428569
	step 10000: train loss 0.0428656, train error 0.00428569
	step 15000: train loss 0.0

# Fashion MNIST

In [13]:
test_model(feature_model=GCNNModelSmall, settings=Baseline_Settings, num_features=10, num_classes=10,
           data=data['fashion_mnist'], feature_samplers=triplet_samplers['fashion_mnist'], classification_samplers=samplers['fashion_mnist'],           
           f_num_steps=60000, f_lr=0.02, f_keep_prob=0.8, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.001)

begin training features // num_features: 10, lr: 0.02, reg: 0.001, dropout: 0.8
	step 0: train loss 2.84762, train error 0.898095
	step 5000: train loss 0.531401, train error 0.0957143
	step 10000: train loss 0.291648, train error 0.0457143
	step 15000: train loss 0.13684, train error 0.00761902
	step 20000: train loss 0.0928604, train error 0.00428569
	step 25000: train loss 0.0619953, train error 0
	step 30000: train loss 0.0510767, train error 0
	step 35000: train loss 0.0449507, train error 0
	step 40000: train loss 0.0413621, train error 0
	step 45000: train loss 0.0384971, train error 0
	step 50000: train loss 0.0364831, train error 0
	step 55000: train loss 0.0346591, train error 0
end training features // time elapsed: 6825.6095 s
test set error: 0.0000 // time elapsed: 0.1499 s
begin training classifier
	step 0: train loss 5.4441, train error 0.855714
	step 5000: train loss 0.00909003, train error 0
	step 10000: train loss 0.00704249, train error 0
	step 15000: train loss 0.00

In [4]:
test_model(feature_model=GCNNModelSmall, settings=Triplet_Hinge_Settings, num_features=10, num_classes=10,
           data=data['fashion_mnist'], feature_samplers=triplet_samplers['fashion_mnist'], classification_samplers=samplers['fashion_mnist'],           
           f_num_steps=60000, f_lr=0.02, f_keep_prob=0.8, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.001)

begin training features // num_features: 10, lr: 0.02, reg: 0.001, dropout: 0.8
	step 0: train loss 1.51831, train error 0.475714
	step 5000: train loss 0.369853, train error 0.0357143
	step 10000: train loss 0.227701, train error 0.0271429
	step 15000: train loss 0.147775, train error 0.0171428
	step 20000: train loss 0.0928849, train error 0.0114286
	step 25000: train loss 0.0671487, train error 0.00714284
	step 30000: train loss 0.0490085, train error 0.0057143
	step 35000: train loss 0.0472363, train error 0.0057143
	step 40000: train loss 0.032361, train error 0.00285715
	step 45000: train loss 0.0367077, train error 0.00285715
	step 50000: train loss 0.0407492, train error 0.00142854
	step 55000: train loss 0.0287412, train error 0
end training features // time elapsed: 6599.2968 s
test set error: 0.0090 // time elapsed: 0.1349 s
begin training classifier
	step 0: train loss 2.8405, train error 0.884286
	step 5000: train loss 0.150761, train error 0.0385714
	step 10000: train los

In [5]:
test_model(feature_model=GCNNModelMedium, settings=Triplet_Hinge_Settings, num_features=10, num_classes=10,
           data=data['fashion_mnist'], feature_samplers=triplet_samplers['fashion_mnist'], classification_samplers=samplers['fashion_mnist'],           
           f_num_steps=60000, f_lr=0.04, f_keep_prob=0.6, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.01, c_reg=0.001)

begin training features // num_features: 10, lr: 0.04, reg: 0.001, dropout: 0.6
	step 0: train loss 1.96235, train error 0.455
	step 5000: train loss 0.443462, train error 0.045
	step 10000: train loss 0.206735, train error 0.035
	step 15000: train loss 0.217126, train error 0.065
	step 20000: train loss 0.164178, train error 0.03
	step 25000: train loss 0.0946572, train error 0.015
	step 30000: train loss 0.0797289, train error 0.015
	step 35000: train loss 0.0894754, train error 0.015
	step 40000: train loss 0.12505, train error 0.045
	step 45000: train loss 0.104694, train error 0.025
	step 50000: train loss 0.0793989, train error 0.005
	step 55000: train loss 0.0997846, train error 0.02
end training features // time elapsed: 2244.5337 s
test set error: 0.0340 // time elapsed: 0.0934 s
begin training classifier
	step 0: train loss 2.64429, train error 0.995
	step 2000: train loss 0.383787, train error 0.11
	step 4000: train loss 0.334439, train error 0.095
	step 6000: train loss 0.3

In [6]:
test_model(feature_model=GCNNModelMedium, settings=Triplet_Hinge_Settings, num_features=10, num_classes=10,
           data=data['fashion_mnist'], feature_samplers=triplet_samplers['fashion_mnist'], classification_samplers=samplers['fashion_mnist'],           
           f_num_steps=50000, f_lr=0.04, f_keep_prob=0.7, f_reg=0.001, 
           c_num_steps=40000, c_lr=0.01, c_reg=0.001)

begin training features // num_features: 10, lr: 0.04, reg: 0.001, dropout: 0.7
	step 0: train loss 1.99959, train error 0.504286
	step 5000: train loss 0.394598, train error 0.0357143
	step 10000: train loss 0.161082, train error 0.0271429
	step 15000: train loss 0.107497, train error 0.03
	step 20000: train loss 0.0648035, train error 0.0142857
	step 25000: train loss 0.0617023, train error 0.0114286
	step 30000: train loss 0.0529438, train error 0.00714284
	step 35000: train loss 0.0467911, train error 0.00285715
	step 40000: train loss 0.0522277, train error 0.00428569
	step 45000: train loss 0.0626263, train error 0.00714284
end training features // time elapsed: 5948.7038 s
test set error: 0.0090 // time elapsed: 0.1406 s
begin training classifier
	step 0: train loss 2.49263, train error 0.922857
	step 5000: train loss 0.150512, train error 0.0471429
	step 10000: train loss 0.176452, train error 0.05
	step 15000: train loss 0.177087, train error 0.0528572
	step 20000: train loss 

In [4]:
test_model(feature_model=GCNNModelMedium, settings=Triplet_Hinge_Settings, num_features=10, num_classes=10,
           data=data['fashion_mnist'], feature_samplers=triplet_samplers['fashion_mnist'], classification_samplers=samplers['fashion_mnist'],           
           f_num_steps=60000, f_lr=0.02, f_keep_prob=0.7, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.001)

begin training features // num_features: 10, lr: 0.02, reg: 0.001, dropout: 0.7
	step 0: train loss 2.02554, train error 0.405714
	step 5000: train loss 0.701622, train error 0.0628572
	step 10000: train loss 0.405892, train error 0.04
	step 15000: train loss 0.26945, train error 0.0428572
	step 20000: train loss 0.176762, train error 0.0271429
	step 25000: train loss 0.105332, train error 0.0157143
	step 30000: train loss 0.0898127, train error 0.0228571
	step 35000: train loss 0.0651041, train error 0.0114286
	step 40000: train loss 0.0592828, train error 0.00857145
	step 45000: train loss 0.0489728, train error 0.0057143
	step 50000: train loss 0.0471318, train error 0.0057143
	step 55000: train loss 0.0515147, train error 0.0057143
end training features // time elapsed: 7095.2122 s
test set error: 0.0120 // time elapsed: 0.1412 s
begin training classifier
	step 0: train loss 2.83233, train error 0.988571
	step 5000: train loss 0.150757, train error 0.03
	step 10000: train loss 0.14